In [1]:
import cv2
import numpy as np

In [2]:
def show_images(imgList):
    while True:
        for (name,img) in imgList:
            cv2.imshow(name,img)
        pressed_key = cv2.waitKey(0)
        if pressed_key==27:
            break
    cv2.destroyAllWindows()

## Gauss Newton

In [3]:
def create_mask(shape,corners,winsize):
    mask = np.zeros(shape[:2],dtype=np.uint8)
    for corner in corners:
        cv2.rectangle(mask,tuple(corner-int(winsize/2)),tuple(corner+int(winsize/2)),255,-1)
    return mask

In [4]:
def get_derivativeof_warp_affine(x,y):
    return np.array([[x,0,y,0,1,0],[0,x,0,y,0,1]])

In [5]:
def gauss_newton_iter(baseimg,endimg,mask,guess):
    assert (baseimg.shape==endimg.shape),"Template and Image should be of same shape"
    endimg = endimg.copy()
    #return baseimg
    
    ## Computing difference T(x)-I(W(x;p))
    transimg = cv2.warpAffine(baseimg,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,3)
    transmask = cv2.warpAffine(mask,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,3)
    diff = np.expand_dims(endimg-transimg,-1) ## (x,y,3,1)
    
    
    ## Calculating gradient and theirs warps (3)
    sobelx = cv2.Sobel(baseimg,cv2.CV_64F,1,0,ksize=3) ## (x,y,3)
    sobely = cv2.Sobel(baseimg,cv2.CV_64F,0,1,ksize=3) ## (x,y,3)
    warped_gradx = cv2.warpAffine(sobelx,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,3)
    warped_grady = cv2.warpAffine(sobely,guess,(endimg.shape[1],endimg.shape[0])) ## (x,y,3)
    warped_grad = np.stack([warped_gradx,warped_grady],-1) ## (x,y,3,2)
    
    ## Caculation Jacobian (4)
    J = np.zeros((baseimg.shape[0],baseimg.shape[1],2,6)) ## (x,y,2,6)
    for x in range(J.shape[0]):
        for y in range(J.shape[1]):
            J[x,y] = get_derivativeof_warp_affine(y,x)

    ## Computing steepest descent images (5)
    steep_desc = np.matmul(warped_grad,J)  ## (x,y,3,6)
    steep_desc[transmask==0] = 0
    steep_desc_trans = steep_desc.transpose((0,1,3,2)) ## (x,y,6,3)
    
    ## Computing hessian (6)
    hessian = np.sum(np.sum(np.matmul(steep_desc_trans,steep_desc),axis=0),axis=0) #(6,6)
    
    ## Computing other term (7)
    other_term = np.sum(np.sum(np.matmul(steep_desc_trans,diff),axis=0),axis=0) #(6,1)
    
    ## calc update param (8)
    inv_hess = np.linalg.inv(hessian) ## (6,6)
    del_p = np.matmul(inv_hess,other_term) ## (6,1)
    
    return del_p.reshape([2,3])

In [28]:
def get_transform(baseimg,endimg,corners):
    baseimg = baseimg.copy()
    endimg = endimg.copy()
    mask = create_mask(baseimg.shape,corners,5)
    #baseimg[mask!=255] = 0
    
    transform = np.float32([[1,0,0],[0,1,0]])
    
    while(True):
        change_param = gauss_newton_iter(baseimg,endimg,mask,transform)
        transform += change_param
        norm = np.linalg.norm(change_param)
        if (norm < 1e-6):
            break
    
    return baseimg

In [61]:
baseimg = right.copy()
endimg = left.copy()
corners = corners.copy()

In [62]:
mask = create_mask(baseimg.shape,corners,5)

In [11]:
transform = np.float32([[1,0,0],[0,1,0]])

In [64]:
transform

array([[  1.01686887e+00,  -8.33746218e-04,   4.01125045e+01],
       [  8.46457657e-03,   1.07577655e+00,   1.58570219e+00]])

In [66]:
for i in range(100):
    change_param = gauss_newton_iter(baseimg,endimg,mask,transform)
    transform += change_param
    print(i)
    print(np.linalg.norm(change_param))
    print(transform)

0
0.0706520215506
[[  1.01776269e+00  -1.01889489e-03   4.01108399e+01]
 [  8.26934196e-03   1.27471747e+00   1.34609944e+00]]
1
0.193123432451
[[  1.01794879e+00  -9.81019064e-04   4.01105822e+01]
 [  8.53843507e-03   1.28671625e+00   1.15334957e+00]]
2
0.0976323238768
[[  1.01806590e+00  -1.01940755e-03   4.01102623e+01]
 [  8.53953385e-03   1.33278738e+00   1.06727167e+00]]
3
0.0468627259014
[[  1.01821630e+00  -1.12485660e-03   4.01099168e+01]
 [  8.33923774e-03   1.37586452e+00   1.08571840e+00]]
4
0.156931004242
[[  1.01836351e+00  -9.18854924e-04   4.01097252e+01]
 [  8.24293568e-03   1.38060511e+00   9.28859362e-01]]
5
0.183916419528
[[  1.01859104e+00  -6.50531201e-04   4.01094344e+01]
 [  8.15564005e-03   1.40126617e+00   7.46107745e-01]]
6
0.0941699071925
[[  1.01892939e+00  -5.74192677e-04   4.01091383e+01]
 [  8.02883599e-03   1.38114338e+00   6.54114155e-01]]
7
0.178333771772
[[  1.01922725e+00  -3.50760731e-04   4.01089850e+01]
 [  7.93210446e-03   1.33408275e+00   4.821

LinAlgError: Singular matrix

## Krunal Pics

In [43]:
best_param = np.float32([[  1.02014029e+00,7.59442290e-03,-2.07362343e-02],
 [  1.89197343e-02,1.32550943e+00,-1.51765833e+01]])

In [7]:
left = cv2.imread("./krunal_pics/room_left.jpg")
right = cv2.imread("./krunal_pics/room_right.jpg")

In [ ]:
show_images([("left",left),("right",right)])

In [49]:
point_left = np.float32([[498,276],[300,186],[561,247],[578,361]])
point_right = np.float32([[451,275],[256,185],[513,245],[531,357]])

In [8]:
gray_right = cv2.cvtColor(right,cv2.COLOR_BGR2GRAY)
corners = cv2.goodFeaturesToTrack(gray_right,25,0.01,10)
corners = np.int0(corners)
corners = np.int0(corners).squeeze()

In [ ]:
affine_transform = get_transform(right,left,corners)

In [44]:
affine_transform = best_param

In [50]:
affine_transform = cv2.getAffineTransform(point_right[:3],point_left[:3])
# persp_transform = cv2.getPerspectiveTransform(point_right,point_left)

In [51]:
affine_transform

array([[  1.01574803e+00,  -7.87401575e-04,   4.01141732e+01],
       [  7.87401575e-03,   9.82939633e-01,   2.14041995e+00]])

In [52]:
left_affine = cv2.warpAffine(right,affine_transform,(right.shape[1],right.shape[0]))
# left_persp = cv2.warpPerspective(right,persp_transform,(right.shape[1]+left.shape[1],right.shape[0]))

In [53]:
#left_affine[0:left.shape[0],0:left.shape[1]] = left
# left_persp[0:left.shape[0],0:left.shape[1]] = left

In [54]:
# show_images([("left",left),("right",right),("affine",left_affine),("persp",left_persp)])

In [55]:
show_images([("left",left),("right",right),("affine",left_affine)])

In [ ]:
for i in corners:
    x,y = i
    cv2.circle(right,(x,y),3,255,-1)